In [1]:
model_name = 'Llama-2-7b-chat-hf'

In [2]:
import json
with open(f'hex-phi/{model_name}.json', 'r', encoding='utf-8') as file:
    harmful_kl_list = json.load(file)
with open(f'sampled_330_alpaca_anchor/{model_name}.json', 'r', encoding='utf-8') as file:
    helpful_kl_list = json.load(file)

In [6]:
helpful_kl_list[0]

{'component_idx': 109,
 'component_type': 'mlp',
 'layer_idx': 1,
 'head_idx': 22,
 'kl': 2704.255569458008}

In [ ]:
component_idx_to_harmful_kl_list = {}
for harmful_kl in harmful_kl_list:
    component_idx = harmful_kl['component_idx']
    component_idx_to_harmful_kl_list[component_idx] = harmful_kl
    
component_idx_to_helpful_kl_list = {}
for helpful_kl in helpful_kl_list:
    component_idx = helpful_kl['component_idx']
    component_idx_to_helpful_kl_list[component_idx] = helpful_kl

KeyError: 109